# 事前データ準備処理

このプロジェクトは、AIにデータ解析してもらう前段階の、データの事前準備（加工）を行うものです。

## 事前準備

In [ ]:
import pandas as pd
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
import math
from tqdm.notebook import tqdm

In [ ]:
# データファイル読み込み
df = pd.read_csv(
    'drive/MyDrive/FX/AI/data/DAT_ASCII_USDJPY_M1_2021.csv',
    sep=';',
    names=('date', 'open', 'high', 'low', 'close', 'spread'),
    converters={'open': Decimal, 'high': Decimal, 'low': Decimal, 'close': Decimal},
    )

In [ ]:
# カラム名を定義
#df = df.set_axis(['date', 'open', 'high', 'low', 'close', 'spread'], axis=1)

In [ ]:
# dateカラムを日付型に変更
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# dateカラムをindex列に指定
df.set_index('date', inplace=True, drop=False)

In [ ]:
df.head()

,date,open,high,low,close,spread,diff
date,,,,,,,
2021-01-03 17:00:00,2021-01-03 17:00:00,103.097000,103.160000,103.097000,103.160000,0,1.0
2021-01-03 17:01:00,2021-01-03 17:01:00,103.161000,103.161000,103.160000,103.161000,0,1.0
2021-01-03 17:02:00,2021-01-03 17:02:00,103.161000,103.197000,103.161000,103.197000,0,1.0
2021-01-03 17:03:00,2021-01-03 17:03:00,103.181000,103.186000,103.181000,103.182000,0,1.0
2021-01-03 17:04:00,2021-01-03 17:04:00,103.184000,103.196000,103.184000,103.195000,0,1.0


## 必要なデータ範囲にカット（過去1年分を目安）

## 欠けている時間があるので補完する

### 次レコードとの差分を取る

In [ ]:
# 割り算を行うときは切り捨て除算("//")演算子を用いている。結果が60秒で割り切れることが明らかなので。
df['diff'] = (df['date'].shift(-1) - df['date']).dt.total_seconds() // 60

### 差が2以上60未満の場合に、欠けているレコードを補完する

In [ ]:
lst_comple = {}

In [ ]:
for index, row in tqdm(df[(1 < df['diff']) & (df['diff'] < 60)].iterrows()):
  # 以下の条件は、2分以上の差があり、60未満の場合補完する、の意味
  # 60未満というのは、非営業日とかだと数日値が動かないので除外している
  # 金曜日の最終時刻で値が動かない場合はもうしょうがない..
  i = 1
  while i < row['diff']:
    newdate = row['date'] + datetime.timedelta(minutes=i)
    # 前の時間の終値から値が動いていないので、すべてcloseで新しい行を作る
    # df.loc[newdate] = [newdate, row.close, row.close, row.close, row.close, row.spread, 1]    # 直接DataFrameに追加するとめちゃくちゃ遅いので、Listに追加していき後でDataFrameに変換して結合する
    lst_comple[newdate] = [newdate, row.close, row.close, row.close, row.close, row.spread, 1]
    i += 1

0it [00:00, ?it/s]

In [ ]:
df_comple = pd.DataFrame.from_dict(lst_comple, orient='index', columns=['date', 'open', 'high', 'low', 'close', 'spread', 'diff'])
del lst_comple
df_comple.head()

,date,open,high,low,close,spread,diff
2021-01-03 17:17:00,2021-01-03 17:17:00,103.212000,103.212000,103.212000,103.212000,0,1
2021-01-03 17:20:00,2021-01-03 17:20:00,103.213000,103.213000,103.213000,103.213000,0,1
2021-01-03 17:23:00,2021-01-03 17:23:00,103.213000,103.213000,103.213000,103.213000,0,1
2021-01-03 17:25:00,2021-01-03 17:25:00,103.213000,103.213000,103.213000,103.213000,0,1
2021-01-04 16:27:00,2021-01-04 16:27:00,103.160000,103.160000,103.160000,103.160000,0,1


In [ ]:
df = pd.concat([df, df_comple])
del df_comple
df.shape

(373173, 7)

In [ ]:
# 行を足したので並べ替え直す
df.sort_index(inplace=True)

## データが1分足なので5分足に調整する

### ダウンサンプリング手法で5分足に丸める(https://note.nkmk.me/python-pandas-resampling-ohlc/)

In [ ]:
d_ohlc = {'open': 'first',
          'high': 'max',
          'low': 'min',
          'close': 'last',}
df = df.resample('5T').agg(d_ohlc)

### 土日も5分足で埋められてしまっているため、NaNが含まれる行は削除する

In [ ]:
df = df.dropna(how='any')

## 各レコードに長期EMA, 中期EMA, 短期EMAのフィールドを追加

In [ ]:
# ewm関数を適用してEMAを追加する
df['Lema'] = df['close'].ewm(span=200, adjust=False).mean()
df['Mema'] = df['close'].ewm(span=75, adjust=False).mean()
df['Sema'] = df['close'].ewm(span=25, adjust=False).mean()

In [ ]:
df['Lema'] = df['Lema'].round(4)
df['Mema'] = df['Mema'].round(4)
df['Sema'] = df['Sema'].round(4)

In [ ]:
df['Lema'] = df['Lema'].map(lambda x: Decimal(str(x)))
df['Mema'] = df['Mema'].map(lambda x: Decimal(str(x)))
df['Sema'] = df['Sema'].map(lambda x: Decimal(str(x)))

## さらに以下を追加

### 長期と中期, 短期, closeとの距離

In [ ]:
df['L_Mdiff'] = df['Lema'] - df['Mema']
df['L_Sdiff'] = df['Lema'] - df['Sema']
df['L_Cdiff'] = df['Lema'] - df['close']

### 中期と短期, closeとの距離

In [ ]:
df['M_Sdiff'] = df['Mema'] - df['Sema']
df['M_Cdiff'] = df['Mema'] - df['close']

### 短期とcloseとの距離

In [ ]:
df['S_Cdiff'] = df['Sema'] - df['close']

### 曜日情報も追加しておきたいかも

In [ ]:
# 0:月曜日、6=日曜日
df['weekday'] = df.index.weekday

### 時刻も入れていいんじゃないかな

## 目的変数を設定する

### 未来5本目のcloseと現在closeとの差額を求める

In [ ]:
df['feature'] = df['close'].shift(-5) - df['close']

In [ ]:
# 最後5行はデータ無しでNaNになるため0埋めする
df = df.fillna(0)

### 差額が+10pips超であれば"1"、以下であれば"0"、-10pips未満であれば"-1"とする

In [ ]:
df['label'] = '0'
df.loc[df['feature'] > 0.05, 'label'] = '1'
df.loc[df['feature'] < -0.05, 'label'] = '-1'

In [ ]:
df['label'].value_counts()

0     59145
1      7859
-1     7623
Name: label, dtype: int64

## 学習データとテストデータに分けて出力する

In [ ]:
# データの分割もNNCがやってくれるので↑は削除
df.to_csv(path_or_buf='drive/MyDrive/FX/AI/data/shaped/DAT_ASCII_USDJPY_M1_2021_CONV.csv', index=False, encoding='utf-8', line_terminator='\r\n')

In [ ]:
df[df.feature < -0.11]

In [ ]:
plt.scatter(df['close'], df['feature'])

In [ ]:
plt.hist(df['label'], bins=50)